In [1]:
using Monkeypox
using Optim
using Optimization
using Plots
using Turing
using StatsPlots
url = "./data/timeseries-country-confirmed.csv"
url1 = "https://raw.githubusercontent.com/globaldothealth/monkeypox/main/timeseries-country-confirmed.csv"
country = "United States"
data_on, acc, cases, datatspan, datadate = datasource!(url, country)

(90×4 DataFrame
 Row │ Date        Cases  Cumulative_cases  Country       
     │ Date        Int64  Int64             String        
─────┼────────────────────────────────────────────────────
   1 │ 2022-05-18      1                 1  United States
   2 │ 2022-05-19      1                 2  United States
   3 │ 2022-05-20      0                 2  United States
   4 │ 2022-05-21      0                 2  United States
   5 │ 2022-05-22      0                 2  United States
   6 │ 2022-05-23      0                 2  United States
   7 │ 2022-05-24      2                 4  United States
   8 │ 2022-05-25      2                 6  United States
  ⋮  │     ⋮         ⋮           ⋮                ⋮
  84 │ 2022-08-09    559              9461  United States
  85 │ 2022-08-10    899             10360  United States
  86 │ 2022-08-11    366             10726  United States
  87 │ 2022-08-12    404             11130  United States
  88 │ 2022-08-13      0             11130  United States
 

In [ ]:
N = 329500000.0
#N = 329500000.0 # population
θ = [0.3, 0.3, 0.2, 0.1, 0.7, 0.01]# ρ,σ,h,α
pknown = [0.0, 0.0, 1 / 30, 1.0] # B,μ,δ,ϕ
lb = [0.0001, 0.0001, 0, 0.0001, 0.0001, 0.0]
ub = [1.0, 1.0, 1.0, 1.0, 1.0, 0.1]
alg = Optim.NelderMead()
p_min = controlmonkeypoxopt!(N, θ, acc, cases, datatspan, pknown, lb, ub, alg)
chainout = controlmonkeypoxinference!(N, p_min, acc, cases, datatspan, pknown, lb, ub)


In [ ]:
println("data parameter:", chainout[2])
prob_pred = controlmonkeypoxprob!(N, θ, acc, pknown)
prediction = controlsimulate!(prob_pred, N, p_min, datatspan, pknown)
scatter(datadate, acc, label="Training data")
display(plot!(datadate, prediction[10, :], label="Real accumulated cases"))
#plot(datadate, prediction[2, :])
display(plot(chainout[1]))